In [1]:
# Importing Required Libraries
import mne
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, SeparableConv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# ------------------- Data Loading and Preprocessing -------------------

# Base path for EEG data files (update this path to your dataset location)
base_path = r'C:\\Users\\mango\\Downloads\\EEG Moter Imagary Datasets\\BCICIV_2a_gdf'
subjects = [f'A0{i}' for i in range(1, 10) if i != 4]  # Exclude Subject 4
event_ids = [7, 8, 9, 10]  # Event IDs: Right Hand, Left Hand, Feet, Tongue

# Initialize lists for features and labels
all_features, all_labels = [], []

# Load data for each subject
for subject in subjects:
    file_path = f'{base_path}/{subject}T.gdf'
    raw = mne.io.read_raw_gdf(file_path, preload=True)
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])  # Drop non-EEG channels
    raw.set_eeg_reference()  # Apply average referencing
    raw.filter(8., 30., fir_design='firwin', verbose=False)  # Bandpass filter (8–30 Hz)

    # Apply ICA for artifact removal
    ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter=800)
    ica.fit(raw)
    raw = ica.apply(raw)

    # Extract events and epochs
    events, _ = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events, event_id=event_ids, tmin=0.5, tmax=4.0, baseline=(0.5, 1.0), preload=True)

    all_features.append(epochs.get_data())  # Shape: (n_epochs, n_channels, n_times)
    all_labels.append(epochs.events[:, -1])  # Event IDs

# Combine all subjects' data
features = np.concatenate(all_features, axis=0)
labels = np.concatenate(all_labels, axis=0)

# Normalize features (Z-score normalization)
normalized_features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

# One-hot encode labels
labels = to_categorical(labels - 7)

# Reshape features for CNN input
normalized_features = normalized_features[..., np.newaxis]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    normalized_features, labels, test_size=0.15, random_state=42, stratify=labels.argmax(axis=1))

# ------------------- Define the EEGNet Model -------------------

model = Sequential()
model.add(Conv2D(16, kernel_size=(1, 64), padding='same', activation='relu',
                 kernel_regularizer=l2(0.01), input_shape=(22, X_train.shape[2], 1)))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(22, 1), depth_multiplier=2, activation='relu', use_bias=False,
                          kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(SeparableConv2D(32, kernel_size=(1, 16), activation='relu', use_bias=False, padding='same',
                          kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# ------------------- Train the Model -------------------

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_split=0.15, callbacks=[reduce_lr, early_stopping], verbose=1)

# ------------------- Evaluate the Model -------------------

# Evaluate model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Set Accuracy: {test_accuracy * 100:.2f}%")

# Generate predictions and evaluation metrics
y_test_pred = np.argmax(model.predict(X_test), axis=1)
y_test_true = np.argmax(y_test, axis=1)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_true, y_test_pred))

print("\nClassification Report:")
print(classification_report(y_test_true, y_test_pred))


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# ------------------- Cross-Validation -------------------

# StratifiedKFold for maintaining class distribution
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
input_shape = (normalized_features.shape[1], normalized_features.shape[2], 1)
num_classes = labels_categorical.shape[1]

accuracy_per_fold = []
confusion_matrices = []

for train_index, val_index in kfold.split(normalized_features, labels):
    # Split data
    X_train, X_val = normalized_features[train_index], normalized_features[val_index]
    y_train, y_val = labels_categorical[train_index], labels_categorical[val_index]

    # Initialize model
    model = create_model(input_shape, num_classes)  # Use the create_model function from the earlier code

    # Define callbacks
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    # Train the model
    print(f"\nTraining fold {fold_no}...")
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val),
              callbacks=[reduce_lr, early_stopping], verbose=1)

    # Evaluate on validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    accuracy_per_fold.append(val_accuracy)
    print(f"Fold {fold_no} Validation Accuracy: {val_accuracy * 100:.2f}%")

    # Predict and calculate confusion matrix
    y_val_pred = np.argmax(model.predict(X_val), axis=1)
    y_val_true = np.argmax(y_val, axis=1)
    confusion_matrices.append(confusion_matrix(y_val_true, y_val_pred))

    # Increment fold number
    fold_no += 1

# ------------------- Results Summary -------------------

print("\nCross-Validation Results:")
for i, acc in enumerate(accuracy_per_fold):
    print(f"Fold {i + 1}: Accuracy = {acc * 100:.2f}%")
print(f"Mean Accuracy: {np.mean(accuracy_per_fold) * 100:.2f}%")
print(f"Standard Deviation: {np.std(accuracy_per_fold) * 100:.2f}%")

# Print confusion matrices
for i, cm in enumerate(confusion_matrices):
    print(f"\nConfusion Matrix for Fold {i + 1}:\n{cm}")
